# Исследование надежности заемщиков.

## Описание проекта.<br>

<b>Заказчик</b> — кредитный отдел банка.<br>
<b>Входные данные от банка</b> — статистика о платёжеспособности клиентов.<br>
<b>Задача исследования -</b> найти зависимость между семейным положением и количеством детей заёмщика и его возможностей погошать кредиты в срок.<br>
<b>Применение результатов исследования -</b> построение модели кредитного скоринга.<br>


### Шаг 1. Обзор данных

In [1]:
import pandas as pd
pd.set_option('display.max_rows', None) # Добавляю условие, для отмены ограничения на показ строк

In [2]:
df = pd.read_csv('data.csv')

In [3]:
df.head(10) # Ознакамливаюсь с данными

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
0,1,-8437.673028,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875.639453,покупка жилья
1,1,-4024.803754,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080.014102,приобретение автомобиля
2,0,-5623.422610,33,Среднее,1,женат / замужем,0,M,сотрудник,0,145885.952297,покупка жилья
3,3,-4124.747207,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628.550329,дополнительное образование
4,0,340266.072047,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616.077870,сыграть свадьбу
5,0,-926.185831,27,высшее,0,гражданский брак,1,M,компаньон,0,255763.565419,покупка жилья
6,0,-2879.202052,43,высшее,0,женат / замужем,0,F,компаньон,0,240525.971920,операции с жильем
7,0,-152.779569,50,СРЕДНЕЕ,1,женат / замужем,0,M,сотрудник,0,135823.934197,образование
8,2,-6929.865299,35,ВЫСШЕЕ,0,гражданский брак,1,F,сотрудник,0,95856.832424,на проведение свадьбы
9,0,-2188.756445,41,среднее,1,женат / замужем,0,M,сотрудник,0,144425.938277,покупка жилья для семьи


In [4]:
df.info() # Получаю информацию о количестве незаполненных строк, узнаю в каких столбцах есть проблемы

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   children          21525 non-null  int64  
 1   days_employed     19351 non-null  float64
 2   dob_years         21525 non-null  int64  
 3   education         21525 non-null  object 
 4   education_id      21525 non-null  int64  
 5   family_status     21525 non-null  object 
 6   family_status_id  21525 non-null  int64  
 7   gender            21525 non-null  object 
 8   income_type       21525 non-null  object 
 9   debt              21525 non-null  int64  
 10  total_income      19351 non-null  float64
 11  purpose           21525 non-null  object 
dtypes: float64(2), int64(5), object(5)
memory usage: 2.0+ MB


Во время предобработки данных мне удалось найти пропуски в столбцах "days_employed" и "total_income".

In [5]:
df.isna().sum() # Подсчитываю количество ячеек с пропусками в столбцах

children               0
days_employed       2174
dob_years              0
education              0
education_id           0
family_status          0
family_status_id       0
gender                 0
income_type            0
debt                   0
total_income        2174
purpose                0
dtype: int64

In [6]:
# Узнаю процентное соотношение количества строк с пропусками к общему количеству строк в столбце days_employed
# Значение также верно для столбца total_income, так как количество пропусков совпадает

print(round(df['days_employed'].isna().sum()/len(df['days_employed']) * 100, 1),'%')

10.1 %


In [7]:
df.isna().mean()

children            0.000000
days_employed       0.100999
dob_years           0.000000
education           0.000000
education_id        0.000000
family_status       0.000000
family_status_id    0.000000
gender              0.000000
income_type         0.000000
debt                0.000000
total_income        0.100999
purpose             0.000000
dtype: float64

**Возможные причины появления пропусков в данных:**
- Человеческий фактор
- Частичная потеря данных по техническим причинам

**Пропуски необходимо заполнить медианным значением, потому что выбросы не смогут сильно исказить данные**

### Шаг 2.1 Заполнение пропусков

In [8]:
df['days_employed'] = df['days_employed'].abs() # Меняю все значения в столбце на положительные числа
days_employed_median = df['days_employed'].median() # Подсчитываю медианное значение в столбце
df['days_employed'] = df['days_employed'].fillna(days_employed_median) # Заполняю пропущенные ячейки медианным значением


total_income_median = df['total_income'].median() # Подсчитываю медианное значение в столбце
df['total_income'] = df['total_income'].fillna(total_income_median) # Заполняю пропущенные ячейки медианным значением 

In [9]:
df.info() # Обращаю внимание на произведённые изменения

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   children          21525 non-null  int64  
 1   days_employed     21525 non-null  float64
 2   dob_years         21525 non-null  int64  
 3   education         21525 non-null  object 
 4   education_id      21525 non-null  int64  
 5   family_status     21525 non-null  object 
 6   family_status_id  21525 non-null  int64  
 7   gender            21525 non-null  object 
 8   income_type       21525 non-null  object 
 9   debt              21525 non-null  int64  
 10  total_income      21525 non-null  float64
 11  purpose           21525 non-null  object 
dtypes: float64(2), int64(5), object(5)
memory usage: 2.0+ MB


In [10]:
df.head(10)

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
0,1,8437.673028,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875.639453,покупка жилья
1,1,4024.803754,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080.014102,приобретение автомобиля
2,0,5623.422610,33,Среднее,1,женат / замужем,0,M,сотрудник,0,145885.952297,покупка жилья
3,3,4124.747207,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628.550329,дополнительное образование
4,0,340266.072047,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616.077870,сыграть свадьбу
5,0,926.185831,27,высшее,0,гражданский брак,1,M,компаньон,0,255763.565419,покупка жилья
6,0,2879.202052,43,высшее,0,женат / замужем,0,F,компаньон,0,240525.971920,операции с жильем
7,0,152.779569,50,СРЕДНЕЕ,1,женат / замужем,0,M,сотрудник,0,135823.934197,образование
8,2,6929.865299,35,ВЫСШЕЕ,0,гражданский брак,1,F,сотрудник,0,95856.832424,на проведение свадьбы
9,0,2188.756445,41,среднее,1,женат / замужем,0,M,сотрудник,0,144425.938277,покупка жилья для семьи


In [11]:
df.isna().sum() # Проверяю, что все пропуски были заполнены 

children            0
days_employed       0
dob_years           0
education           0
education_id        0
family_status       0
family_status_id    0
gender              0
income_type         0
debt                0
total_income        0
purpose             0
dtype: int64

In [12]:
df.head(10)

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
0,1,8437.673028,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875.639453,покупка жилья
1,1,4024.803754,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080.014102,приобретение автомобиля
2,0,5623.422610,33,Среднее,1,женат / замужем,0,M,сотрудник,0,145885.952297,покупка жилья
3,3,4124.747207,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628.550329,дополнительное образование
4,0,340266.072047,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616.077870,сыграть свадьбу
5,0,926.185831,27,высшее,0,гражданский брак,1,M,компаньон,0,255763.565419,покупка жилья
6,0,2879.202052,43,высшее,0,женат / замужем,0,F,компаньон,0,240525.971920,операции с жильем
7,0,152.779569,50,СРЕДНЕЕ,1,женат / замужем,0,M,сотрудник,0,135823.934197,образование
8,2,6929.865299,35,ВЫСШЕЕ,0,гражданский брак,1,F,сотрудник,0,95856.832424,на проведение свадьбы
9,0,2188.756445,41,среднее,1,женат / замужем,0,M,сотрудник,0,144425.938277,покупка жилья для семьи


### Шаг 2.2 Проверка данных на аномалии и исправления.

### Шаг 2.3. Изменение типов данных.

In [13]:
df['total_income'] = df['total_income'].astype(int)

### Шаг 2.4. Удаление дубликатов.

In [14]:
df.duplicated().sum() # Нахожу количество неявных дубликатов

54

In [15]:
df = df.drop_duplicates().reset_index(drop = True) # Удаляю неявные дубликаты

In [16]:
df.duplicated().sum() # Проверяю, что неявных дубликатов не осталось

0

In [17]:
# Исправление данных в столбце education:
df['education'].sort_values().unique() # Нахожу все уникальные значения в столбце

array(['ВЫСШЕЕ', 'Высшее', 'НАЧАЛЬНОЕ', 'НЕОКОНЧЕННОЕ ВЫСШЕЕ',
       'Начальное', 'Неоконченное высшее', 'СРЕДНЕЕ', 'Среднее',
       'УЧЕНАЯ СТЕПЕНЬ', 'Ученая степень', 'высшее', 'начальное',
       'неоконченное высшее', 'среднее', 'ученая степень'], dtype=object)

In [18]:
df['education'] = df['education'].str.lower() # Привожу все значения в столбцу к единому стилю

In [19]:
df['education'].sort_values().unique() # Проверяю, что все значения были приведены к единому стилю

array(['высшее', 'начальное', 'неоконченное высшее', 'среднее',
       'ученая степень'], dtype=object)

**Проверяю другие столбцы на возможность присутствия аномалий**

In [20]:
df['family_status'].sort_values().unique() # Аномалии отсутствуют

array(['Не женат / не замужем', 'в разводе', 'вдовец / вдова',
       'гражданский брак', 'женат / замужем'], dtype=object)

In [21]:
df['income_type'].sort_values().unique() # Аномалии отсутствуют

array(['безработный', 'в декрете', 'госслужащий', 'компаньон',
       'пенсионер', 'предприниматель', 'сотрудник', 'студент'],
      dtype=object)

In [22]:
df['children'].sort_values().unique() 

array([-1,  0,  1,  2,  3,  4,  5, 20])

In [23]:
df.groupby(['children']).size()

children
-1        47
 0     14107
 1      4809
 2      2052
 3       330
 4        41
 5         9
 20       76
dtype: int64

**Есть аномалии.**

Значение в столбце не может принимать значения "-1" (47 строк с таким значением)<br>
Также высока вероятность человеческой ошибки во время ввода значения "20" (76 строк с таким значением).

Значение "20" необходимо заменить на "2"<br>
Значение "-1" необходимо заменить на "1"

In [24]:
df['children'] = df['children'].replace(-1, 1)
df['children'] = df['children'].replace(20, 2)

In [25]:
# Проверяю, что всё аномалии были заменены на корректные значения
df['children'].sort_values().unique()

array([0, 1, 2, 3, 4, 5])

In [26]:
df['purpose'].sort_values().unique() # Необходимо разбить эти значения по нескольким категориям для удобства работы

array(['автомобили', 'автомобиль', 'высшее образование',
       'дополнительное образование', 'жилье',
       'заняться высшим образованием', 'заняться образованием',
       'на покупку автомобиля', 'на покупку подержанного автомобиля',
       'на покупку своего автомобиля', 'на проведение свадьбы',
       'недвижимость', 'образование', 'операции с жильем',
       'операции с коммерческой недвижимостью',
       'операции с недвижимостью', 'операции со своей недвижимостью',
       'покупка жилой недвижимости', 'покупка жилья',
       'покупка жилья для сдачи', 'покупка жилья для семьи',
       'покупка коммерческой недвижимости', 'покупка недвижимости',
       'покупка своего жилья', 'получение высшего образования',
       'получение дополнительного образования', 'получение образования',
       'приобретение автомобиля', 'профильное образование',
       'ремонт жилью', 'свадьба', 'свой автомобиль',
       'сделка с автомобилем', 'сделка с подержанным автомобилем',
       'строительство 


### Шаг 2.5. Формирование дополнительных датафреймов словарей, декомпозиция исходного датафрейма.

In [27]:
# Необходимо сопоставить данные из столбцов 

df_1 = df.loc[:, ['education', 'education_id']] #
display(df_1.drop_duplicates().reset_index(drop = True))


,education,education_id
0,высшее,0
1,среднее,1
2,неоконченное высшее,2
3,начальное,3
4,ученая степень,4


In [28]:
# Создаю словарь 
education_dictionary = pd.Series(df_1.education.values,index=df_1.education_id).to_dict()
education_dictionary

{0: 'высшее',
 1: 'среднее',
 2: 'неоконченное высшее',
 3: 'начальное',
 4: 'ученая степень'}

In [29]:
# Необходимо сопоставить данные из столбцов 

df_2 = df.loc[:, ['family_status', 'family_status_id']]
df_2 = df_2.drop_duplicates().reset_index(drop = True)
display(df_2)

,family_status,family_status_id
0,женат / замужем,0
1,гражданский брак,1
2,вдовец / вдова,2
3,в разводе,3
4,Не женат / не замужем,4


In [30]:
# Создаю словарь 
family_status_dictionary = pd.Series(df_2.family_status.values,index=df_2.family_status_id).to_dict()
family_status_dictionary

{0: 'женат / замужем',
 1: 'гражданский брак',
 2: 'вдовец / вдова',
 3: 'в разводе',
 4: 'Не женат / не замужем'}

### Шаг 2.6. Категоризация дохода.

In [31]:
def apply_income_category(row):
    
    total_income = row['total_income']
    
    if total_income < 30000:
        return 'E'
    if total_income < 50000:
        return 'D'
    if total_income < 200000:
        return 'C'
    if total_income < 1000000:
        return 'B'
    if total_income > 1000000:
        return 'A'
    
    return 'категория не определена'

df['income_category'] = df.apply(apply_income_category, axis=1)

In [32]:
df.head() # Проверяю, что функция отработала

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose,income_category
0,1,8437.673028,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875,покупка жилья,B
1,1,4024.803754,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080,приобретение автомобиля,C
2,0,5623.422610,33,среднее,1,женат / замужем,0,M,сотрудник,0,145885,покупка жилья,C
3,3,4124.747207,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628,дополнительное образование,B
4,0,340266.072047,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616,сыграть свадьбу,C


### Шаг 2.7. Категоризация целей кредита.

In [33]:
def change_purpose(row):
    
    purpose = row['purpose']
    
    if 'авто' in purpose:
        return 'операции с автомобилем'
    if 'образов' in purpose:
        return 'получение образования'
    if 'свадьб' in purpose:
        return 'проведение свадьбы'
    if 'недвиж' in purpose:
        return 'операции с недвижимостью'
    if 'жиль' in purpose:
        return 'операции с недвижимостью'
    
    return 'категория не определена'

df['purpose_category'] = df.apply(change_purpose, axis=1)

In [34]:
df.head() # Проверяю, что функция отработала

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose,income_category,purpose_category
0,1,8437.673028,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875,покупка жилья,B,операции с недвижимостью
1,1,4024.803754,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080,приобретение автомобиля,C,операции с автомобилем
2,0,5623.422610,33,среднее,1,женат / замужем,0,M,сотрудник,0,145885,покупка жилья,C,операции с недвижимостью
3,3,4124.747207,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628,дополнительное образование,B,получение образования
4,0,340266.072047,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616,сыграть свадьбу,C,проведение свадьбы


### Ответы на вопросы.


##### Вопрос 1:
**Есть ли зависимость между количеством детей и возвратом кредита в срок?**<br>

In [35]:
df['children'].sort_values().unique()

array([0, 1, 2, 3, 4, 5])

In [36]:
pivot_children_debt = pd.pivot_table(df,index=['children'],values=['debt'], aggfunc=['sum','count', 'mean'], margins=True)
pivot_children_debt

,sum,count,mean
,debt,debt,debt
children,,,
0,1063,14107,0.075353
1,445,4856,0.091639
2,202,2128,0.094925
3,27,330,0.081818
4,4,41,0.097561
5,0,9,0.000000
All,1741,21471,0.081086


## Вывод 1:

Трудно говорить о какой-либо зависимости между колчеством детей в семье и вероятностью выплаты долга в срок. Средняя доля неплательщиков в каждой из групп составляет в среднем 8%. (Семьи с 5 детьми не стоит рассматривать, так как таких семей всего 9 в нашей выборке. Этим можно пренебречь.

##### Вопрос 2:
**Есть ли зависимость между семейным положением и возвратом кредита в срок?**

In [37]:
print(df['family_status'].sort_values().unique())
pivot_children_debt = pd.pivot_table(df,index=['family_status'],values=['debt'], aggfunc=['sum','count', 'mean'], margins=True)
pivot_children_debt

['Не женат / не замужем' 'в разводе' 'вдовец / вдова' 'гражданский брак'
 'женат / замужем']


,sum,count,mean
,debt,debt,debt
family_status,,,
Не женат / не замужем,274,2810,0.097509
в разводе,85,1195,0.071130
вдовец / вдова,63,959,0.065693
гражданский брак,388,4163,0.093202
женат / замужем,931,12344,0.075421
All,1741,21471,0.081086


##### Вывод 2:
Сильной корреляции между семейным положением и возврата кредита в срок нельзя увидеть. Можно заметить, что наиболишее количество неплательщиков находятся в группе "Не женат / не замужем" (около 10 процентов). Наиболее ответственно к своим обязательствам относятся заёмщики из группы "вдовец / вдова".

##### Вопрос 3:
**Есть ли зависимость между уровнем дохода и возвратом кредита в срок?**<br>

In [38]:
print(df['income_category'].sort_values().unique())
pivot_children_debt = pd.pivot_table(df,index=['income_category'],values=['debt'], aggfunc=['sum','count', 'mean'], margins=True)
pivot_children_debt

['A' 'B' 'C' 'D' 'E']


,sum,count,mean
,debt,debt,debt
income_category,,,
A,2,25,0.080000
B,356,5041,0.070621
C,1360,16033,0.084825
D,21,350,0.060000
E,2,22,0.090909
All,1741,21471,0.081086


##### Вывод 3:
Подавляющее число заёмщиков находятся в категориях "В" и "С". Количество должников, которые не возвращают кредит в срок порядка 7-8 процентов. Граждане из категории "D" более ответственно относятся к выплате своих обязательств в срок. Процент невозврата кредитов среди заёмщиков с доходом более одного миллиона рублей выши остальных групп (около 9 процентов, правда, в данную группу входит только 22 заёмщика). 

##### Вопрос 4:
**Как разные цели кредита влияют на его возврат в срок?**<br>

In [39]:
print(df['purpose_category'].sort_values().unique())
pivot_children_debt = pd.pivot_table(df,index=['purpose_category'],values=['debt'], aggfunc=['sum','count', 'mean'], margins=True)
pivot_children_debt

['операции с автомобилем' 'операции с недвижимостью'
 'получение образования' 'проведение свадьбы']


,sum,count,mean
,debt,debt,debt
purpose_category,,,
операции с автомобилем,403,4308,0.093547
операции с недвижимостью,782,10814,0.072314
получение образования,370,4014,0.092177
проведение свадьбы,186,2335,0.079657
All,1741,21471,0.081086


##### Вывод 4:
Наиболее ответственно к выплате кредитов в срок относятся заёмщики, которые берут кредит с целью покупки недвижимости. (Процент невозвратов около 7 процентов). Реже отдают кредиты в срок люди, которые брали кредит на свадьбу (около 8 процентов). Наибольшие проблемы с возвратом наблюдаются в группах людей, которые брали кредит на покупку автомобиля, либо на получение образования (более 9 процентов)

## Общий вывод:

В ходе исследования мне удалось установить, что:

1. Трудно говорить о какой-либо зависимости между колчеством детей в семье и вероятностью выплаты долга в срок. 
2. Сильной корреляции между семейным положением и возврата кредита в срок нельзя увидеть. Можно заметить, что наибольшее количество неплательщиков находятся в группе "Не женат / не замужем" (около 10 процентов). Наиболее ответственно к своим обязательствам относятся заёмщики из группы "вдовец / вдова" (около 6 процентов).
3. Подавляющее число заёмщиков находятся в категориях "В" и "С" (люди, уровень дохода которых от 50тр до 1млн). Количество должников, которые не возвращают кредит в срок порядка 7-8 процентов. Граждане из категории "D" более ответственно относятся к выплате своих обязательств в срок. Процентом невозврата кредитов среди наболее бедной и богатой групп можно пренебречь, так как в эту выборку входит очень малое количество заёмщиков.
4. Наиболее ответственно к выплате кредитов в срок относятся заёмщики, которые берут кредит с целью покупки недвижимости. (Процент невозвратов около 7 процентов). Реже отдают кредиты в срок люди, которые брали кредит на свадьбу (около 8 процентов). Наибольшие проблемы с возвратом наблюдаются в группах людей, которые брали кредит на покупку автомобиля, либо на получение образования (более 9 процентов)

